In [25]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fuzzywuzzy import process

/Users/philipdecanesie/.local/share/virtualenvs/Machine-learning-Philip-de-Canesie-tsBDOfXA/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
movies_path = "../Data/ml-latest-small/movies.csv"
ratings_path = "../Data/ml-latest-small/ratings.csv"

df_movies = pd.read_csv(movies_path, usecols=["movieId", "title"], dtype= {"movieId": "int32", "title": "str"})
df_ratings = pd.read_csv(ratings_path, usecols=["userId", "movieId", "rating"], dtype= {"userId": "int32", "movieId": "int32", "rating": "float32"})

In [11]:
# Merge the dataframes
movie_ratings = pd.merge(df_ratings, df_movies, on='movieId')

# Create a pivot table of user ratings
user_ratings = pd.pivot_table(movie_ratings, index='title', columns='userId', values='rating')

# Fill missing values with 0
user_ratings = user_ratings.fillna(0)

# Fit the model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_ratings.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [47]:
# Function to get recommendations
def get_recommendations(movie_name, n_recommendations):
    movie_title = process.extractOne(movie_name, user_ratings.index)[0]
    movie_idx = np.where(user_ratings.index == movie_title)[0][0]
    print(movie_title)
    distances, indices = model_knn.kneighbors(user_ratings.values[movie_idx].reshape(1, -1), n_neighbors=n_recommendations+1)
    recommended_movies = []
    for i in range(1, len(distances.flatten())):
        recommended_movies.append(user_ratings.index[indices.flatten()[i]])
    return recommended_movies

# Example usage
recommended_movies = get_recommendations('toy story', 10)


for title in recommended_movies:
    print(title)

Pirates of Silicon Valley (1999)
Getaway, The (1972)
Pit and the Pendulum (1961)
Antitrust (2001)
Skipped Parts (2000)
Invincible (2001)
Things You Can Tell Just by Looking at Her (2000)
Ordinary Decent Criminal (2000)
Scotland, Pa. (2001)
Mondays in the Sun (Lunes al sol, Los) (2002)
Love and a .45 (1994)


In [41]:
movie_title = process.extractOne("toy story", user_ratings.index)[0]
movie_idx = np.where(user_ratings.index == movie_title)[0][0]
print(movie_idx)

8871
